# Classification Model

## Training/Testing/Validation of Time-series based model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
from sklearn.metrics import confusion_matrix
import xgboost as xgb

In [ ]:
binary_target = 'some_target'
features = ['f1','f2','f3']
train_df = df.copy()
train_df[binary_target] = np.where(train_df[binary_target]>tgt,1,0)

In [ ]:
#withhold future time-series data to avoid overfitting
val_size = int(train_df.shape[0]*0.10)
validation_set = train_df[:val_size]
train_df = train_df[val_size:]

In [ ]:
print(train_df.shape)

train_cols = features
train_x = train_df[train_cols]
train_y = train_df[binary_target] #['cst_unified_above_tgt']

train_size = int(train_x.shape[0]*0.6)
X_train, X_test, y_train, y_test = train_x[:train_size], train_x[train_size:]\
                , train_y[:train_size],  train_y[train_size:]

#arbitrary hyperparameters, could optimize with grid/random search
xgb_clf = xgb.XGBClassifier(learning_rate=0.01, 
                            max_depth=4, 
                            colsample_bytree=0.5,
                            #missing=0,
                            subsample=0.8,
                            n_estimators=100000,
                            scale_pos_weight=1.1,
                           gamma=2)



xgb_clf.fit(X_train, y_train, eval_set=[(X_train,y_train),(X_test, y_test)], eval_metric='auc', early_stopping_rounds=150, verbose=0)

y_pred = xgb_clf.predict(X_test)

## Validation

In [ ]:
#report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=['tgt<60%','tgt>60%']))

In [ ]:
import seaborn as sns
#confusion matrix
matrix = confusion_matrix(y_test,y_pred)

#pandas dataframe
dfc = pd.DataFrame(matrix)
dfc = dfc.rename({0:'tgt<60%',1:'tgt>60%'},axis='index')
dfc = dfc.rename({0:'Predicted_tgt<60%',1:'Predicted_tgt>60%'},axis='columns')

#heatmap
sns.heatmap(dfc,annot=True,cbar=None,cmap="Blues",fmt='g')
plt.ylabel('Real Values')
plt.xlabel('predicted Values')

### repeat the validation above for the validation set to ensure no overfitting
### can also pair with cross validation